In [124]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from sklearn.model_selection import train_test_split

import torchvision as tv

import os
import numpy as np
import matplotlib.pyplot as plt
from tqdm.autonotebook import tqdm

from torch.cuda.amp import autocast, GradScaler
import plotly.graph_objects as go
import seaborn as sns
import plotly.express as px
import pandas as pd

In [125]:
X_array =  np.load('X_array.npy')
Y_array =  np.load('Y_array.npy')


In [126]:
X_train, X_test, y_train, y_test = train_test_split(X_array, Y_array, test_size=0.2, random_state=898)

In [127]:
X_train = torch.tensor(X_train)
y_train = torch.tensor(y_train)
X_test = torch.tensor(X_test)
y_test = torch.tensor(y_test)

In [128]:
print(len(X_train))
print(len(y_train))
print(len(X_test))
print(len(y_test))

359
359
90
90


In [129]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

In [130]:
# Создаем Dataset и DataLoader
dataset_train = CustomDataset(X_train, y_train)
dataset_test = CustomDataset(X_test, y_test)
dataloader_train = torch.utils.data.DataLoader(dataset_train, batch_size=5, shuffle=True)
dataloader_test = torch.utils.data.DataLoader(dataset_test, batch_size=5, shuffle=False)

In [131]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        # Определение сверточных слоев
        self.conv1 = nn.Conv2d(1, 16, kernel_size=4, stride=1, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        #self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        # Определение пулинга
        self.pool = nn.MaxPool2d(kernel_size=3, stride=2, padding=0)
        # Определение полносвязных слоев
        self.fc1 = nn.Linear(32*24*10, 128)  # 128 каналов, 32x25 после применения пулинга
        self.fc2 = nn.Linear(128, 10)  # 10 классов

    def forward(self, x):
        # Пропускание данных через сверточные слои и функции активации
        x = self.pool(F.leaky_relu(self.conv1(x)))
        x = self.pool(F.leaky_relu(self.conv2(x)))
        #x = self.pool(F.leaky_relu(self.conv3(x)))
    
        # Выпрямление данных перед подачей на полносвязный слой
        x = x.view(-1, 32*24*10)  # Выпрямляем тензор
        x = F.leaky_relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Создание экземпляра нейронной сети
net = ConvNet()
# Печать архитектуры нейронной сети
print(net)

ConvNet(
  (conv1): Conv2d(1, 16, kernel_size=(4, 4), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=7680, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)


In [132]:
for param in net.parameters():
    param.data = param.data.double()

In [133]:
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.0001, betas=(0.9, 0.999))
scheduler = torch.optim.lr_scheduler.ExponentialLR(
    optimizer,
    gamma = 0.6
)

In [134]:
def accuracy(pred, label, threshold=0.5):
    pred = torch.sigmoid(pred)
    pred_labels = (pred > threshold).float()  # Threshold the predictions
    correct = (pred_labels == label).sum().item()  # Compare predictions with labels
    total = label.size(0) * label.size(1)  # Total number of labels
    return correct / total


In [135]:
device = 'cuda' # if torch.cuda.is_available() else 'cpu'
model = net.to(device)
loss_fn = loss_fn.to(device)

In [136]:
use_amp = True
scaler = torch.cuda.amp.GradScaler()
torch.backends.cudnn.benchmark = True
torch.backends.cudnn.deterministic = False

In [137]:
epochs = 100

for epoch in range(epochs):
    loss_val = 0.0
    acc_val = 0.0
    for sample in tqdm(dataloader_train):
        matrix, label = sample[0].to(device), sample[1].to(device)
        optimizer.zero_grad()

        # Добавим измерение каналов к данным
        matrix = matrix.unsqueeze(1)
  
        with autocast(use_amp):
            pred = model(matrix)
            loss = loss_fn(pred, label.float())

        scaler.scale(loss).backward()
        loss_item = loss.item()
        loss_val += loss_item

        scaler.step(optimizer)
        scaler.update()

        acc_current = accuracy(pred.cpu().float(), label.cpu().float())
        acc_val += acc_current

    print(f'Epoch: [{epoch+1}/{epochs}], Loss: {loss_val/len(dataloader_train):.5f}, Accuracy: {acc_val/len(dataloader_train):.3f}')

100%|██████████| 72/72 [00:00<00:00, 92.27it/s] 


Epoch: [1/100], Loss: 0.74712, Accuracy: 0.701


100%|██████████| 72/72 [00:00<00:00, 151.68it/s]


Epoch: [2/100], Loss: 0.57961, Accuracy: 0.770


100%|██████████| 72/72 [00:00<00:00, 149.33it/s]


...

Epoch: [98/100], Loss: 0.24963, Accuracy: 0.891


100%|██████████| 72/72 [00:00<00:00, 156.80it/s]


Epoch: [99/100], Loss: 0.25264, Accuracy: 0.892


100%|██████████| 72/72 [00:00<00:00, 154.39it/s]

Epoch: [100/100], Loss: 0.25746, Accuracy: 0.887


In [138]:
loss_val = 0.0
acc_val = 0.0
for sample in tqdm(dataloader_test):
    matrix, label = sample[0].to(device), sample[1].to(device)

    # Добавим измерение каналов к данным
    matrix = matrix.unsqueeze(1)
  
    with autocast(use_amp):
        pred = model(matrix)
        loss = loss_fn(pred, label)

    loss_item = loss.item()
    loss_val += loss_item


    acc_current = accuracy(pred.cpu().float(), label.cpu().float())
    acc_val += acc_current


print(f'Loss: {loss_val/len(dataloader_test):.5f}, Accuracy: {acc_val/len(dataloader_test):.3f}')

100%|██████████| 18/18 [00:00<00:00, 428.59it/s]

Loss: 0.86090, Accuracy: 0.788


In [139]:
def aroma_map(matrix):
    matrix = matrix.unsqueeze(1)
    pred = model(matrix).to('cpu')


    df = pd.DataFrame(dict(
        r=torch.sigmoid(pred).detach().numpy()[0],
        theta=['Herbs and spices', 'Tobacco/Smoke', 'Wood', 'Berries', 'Citrus',
       'Fruits ', 'Nuts', 'Coffee', 'Chocolate/Cacao', 'Flowers']))
    
    fig = px.line_polar(df, r='r', theta='theta', line_close=True, template="plotly_dark")
    fig.update_traces(fill='toself', line_color='maroon')
    fig.update_layout(polar=dict(radialaxis=dict(range=[0, 1])))
    fig.show()


In [140]:
aroma_map(X_train[42].to(device).unsqueeze(0))

In [141]:
def aroma_map_comparison(matrix, labels):
    matrix = matrix.unsqueeze(1)
    pred = model(matrix).to('cpu')
    labels = labels.to('cpu')

    print(labels)

    df = pd.DataFrame(dict(
        r=torch.sigmoid(pred).detach().numpy()[0],
        theta=['Herbs and spices', 'Tobacco/Smoke', 'Wood', 'Berries', 'Citrus',
       'Fruits ', 'Nuts', 'Coffee', 'Chocolate/Cacao', 'Flowers']))
    
    df['Label'] = 'Predict'
    
    df1 = pd.DataFrame(dict(
        r=labels.detach().numpy(),
        theta=['Herbs and spices', 'Tobacco/Smoke', 'Wood', 'Berries', 'Citrus',
       'Fruits ', 'Nuts', 'Coffee', 'Chocolate/Cacao', 'Flowers']))
    
    df1['Label'] = 'Experiment'
    
    fig = px.line_polar(pd.concat([df, df1]), color="Label",  r='r', theta='theta', line_close=True, line_shape='linear', color_discrete_sequence=['#008080', '#FFC0CB'],
                    template="plotly_dark")
    

    
    fig.update_traces(fill='toself')
    fig.update_layout(polar=dict(radialaxis=dict(visible=True, range=[0, 1])))
    fig.show()

In [142]:
for i in range(90):
    matrix, label = X_test[i].to(device).unsqueeze(0), y_test[i].to(device)
    aroma_map_comparison(matrix, label)

tensor([0., 0., 0., 0., 1., 1., 0., 0., 0., 1.], dtype=torch.float64)


tensor([1., 0., 0., 0., 0., 1., 0., 0., 0., 1.], dtype=torch.float64)


tensor([0., 0., 0., 0., 0., 1., 1., 0., 0., 1.], dtype=torch.float64)


tensor([0., 0., 0., 1., 0., 1., 0., 0., 0., 0.], dtype=torch.float64)


tensor([1., 0., 1., 1., 0., 0., 0., 0., 1., 0.], dtype=torch.float64)


...

tensor([1., 0., 0., 0., 1., 1., 0., 0., 0., 1.], dtype=torch.float64)


tensor([1., 0., 0., 1., 0., 1., 1., 0., 0., 1.], dtype=torch.float64)


tensor([1., 0., 0., 1., 0., 0., 0., 0., 0., 1.], dtype=torch.float64)


tensor([1., 0., 0., 0., 0., 1., 0., 0., 0., 0.], dtype=torch.float64)


tensor([0., 0., 0., 0., 1., 1., 0., 0., 0., 0.], dtype=torch.float64)
